In [7]:
# -*- coding: utf-8 -*-
import os
import re
import sys
import arcpy
from samgeo import SamGeo2
import traceback

# 新增：釋放記憶體需要
import gc
try:
    import torch
except ImportError:
    torch = None

# ========= 使用者需確認/可調參數 =========
ortho_dir      = r"D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\ortho_splits"   # 正射影像資料夾
points_gdb     = r"D:\ArcGIS\AI_training\split_points.gdb"                        # 點圖層所在 GDB (含 T1, T2,...)
mask_dir       = r"D:\ArcGIS\AI_training\masks"                                   # 二值遮罩輸出資料夾 (GeoTIFF)
poly_gdb       = r"D:\ArcGIS\AI_training\geosam_output.gdb"                       # RasterToPolygon 輸出 GDB
filtered_gdb   = r"D:\ArcGIS\AI_training\shp_filtered.gdb"                        # SpatialJoin/過濾/填洞輸出 GDB

model_id       = "sam2-hiera-small"
small_area_threshold = 0.00001    # 刪除過小面積門檻
hole_area_threshold  = 0.0005     # EliminatePolygonPart 填洞門檻
# =====================================

arcpy.env.overwriteOutput = True

# 確保輸出容器存在
os.makedirs(mask_dir, exist_ok=True)
for gdb in [poly_gdb, filtered_gdb]:
    if not arcpy.Exists(gdb):
        arcpy.management.CreateFileGDB(os.path.dirname(gdb),
                                       os.path.splitext(os.path.basename(gdb))[0])

# 讀取ArcGIS Pro專案(可選，主要用來印環境資訊)
try:
    aprx = arcpy.mp.ArcGISProject("CURRENT")
    print("專案路徑：", aprx.filePath)
    print("預設 gdb：", aprx.defaultGeodatabase)
    print("專案資料夾：", aprx.homeFolder)
    if aprx.activeMap:
        print("目前 Map 名稱:", aprx.activeMap.name)
        sr = aprx.activeMap.spatialReference
        if sr:
            print("Map 空間參考:", sr.name, sr.factoryCode)
except Exception as e:
    print("取得 ArcGIS Project 資訊時發生非致命錯誤：", e)

print("Python 可執行檔：", sys.executable)
print("Conda 環境路徑：", sys.prefix)

# 掃描影像：ortho_split_XX.tif，擷取序號
ortho_pat = re.compile(r"^ortho_split_(\d{2})\.tif$", re.IGNORECASE)
ortho_map = {}  # {'01': full_path, ...}
for fn in os.listdir(ortho_dir):
    m = ortho_pat.match(fn)
    if m:
        key = m.group(1)  # 兩位數序號字串
        ortho_map[key] = os.path.join(ortho_dir, fn)

# 掃描點圖層：T1、T2…（支援無前導零）
point_map = {}  # {'01': featureclass_path, ...}
arcpy.env.workspace = points_gdb
for fc in arcpy.ListFeatureClasses():
    m = re.match(r"^T(\d+)$", fc, re.IGNORECASE)
    if m:
        n = int(m.group(1))
        key = f"{n:02d}"
        point_map[key] = os.path.join(points_gdb, fc)

# 取兩者交集序號（同時存在的配對才處理）
common_keys = sorted(set(ortho_map.keys()) & set(point_map.keys()))
if not common_keys:
    raise RuntimeError("找不到任何影像與點圖層的對應配對，請檢查檔名/圖層名規則。")

print("將處理以下配對序號：", common_keys)

# 🔥 移除：原本的「初始化 SAM 模型（只建立一次）」——你要每回合各自建，避免狀態殘留
# sam = SamGeo2(model_id=model_id, automatic=False)

def get_point_coords_and_crs(points_fc):
    """回傳 (coords_list, point_crs_str, epsg_int or None)"""
    coords = []
    with arcpy.da.SearchCursor(points_fc, ["SHAPE@X", "SHAPE@Y"], where_clause="Shape IS NOT NULL") as cur:
        for x, y in cur:
            coords.append([x, y])
    sr = arcpy.Describe(points_fc).spatialReference
    epsg = sr.factoryCode
    if epsg and int(epsg) > 0:
        point_crs = f"EPSG:{int(epsg)}"
    else:
        point_crs = sr.exportToString()
    return coords, point_crs, (int(epsg) if epsg else None)

def raster_to_polygon(in_ras, out_gdb, out_name):
    out_fc = os.path.join(out_gdb, out_name)
    arcpy.conversion.RasterToPolygon(in_ras, out_fc)  # 保持你原本的呼叫樣式
    return out_fc

def spatial_join_keep_polys(polys_fc, points_fc, out_fc):
    fm = arcpy.FieldMappings()
    fm.addTable(polys_fc)
    arcpy.analysis.SpatialJoin(
        target_features=polys_fc,
        join_features=points_fc,
        out_feature_class=out_fc,
        join_operation="JOIN_ONE_TO_ONE",
        join_type="KEEP_COMMON",
        field_mapping=fm,
        match_option="CONTAINS"
    )

def delete_small_and_largest(out_fc, small_thresh):
    oid_field = arcpy.Describe(out_fc).oidFieldName
    max_oid, max_area = None, float("-inf")
    with arcpy.da.SearchCursor(out_fc, [oid_field, "SHAPE@AREA"]) as cur:
        for oid, area in cur:
            if area is not None and area > max_area:
                max_area = area
                max_oid = oid
    deleted_small = 0
    with arcpy.da.UpdateCursor(out_fc, [oid_field, "SHAPE@AREA"]) as ucur:
        for oid, area in ucur:
            if area is not None and area < small_thresh:
                ucur.deleteRow()
                deleted_small += 1
    deleted_max = 0
    if max_oid is not None:
        with arcpy.da.UpdateCursor(out_fc, [oid_field]) as ucur2:
            for oid, in ucur2:
                if oid == max_oid:
                    ucur2.deleteRow()
                    deleted_max = 1
                    break
    return deleted_small, deleted_max

def eliminate_holes(in_fc, out_fc, hole_thresh):
    arcpy.management.EliminatePolygonPart(
        in_features=in_fc,
        out_feature_class=out_fc,
        condition="AREA",
        part_area=hole_thresh,
        part_option="CONTAINED_ONLY"
    )

# ===== 主迴圈 =====
for key in common_keys:
    ortho = ortho_map[key]
    points_fc = point_map[key]
    print("\n========== 開始處理序號", key, "==========")
    print("影像：", ortho)
    print("點圖層：", points_fc)

    sam = None  # for safety: 保證 finally 可以刪除
    try:
        coords, point_crs, epsg = get_point_coords_and_crs(points_fc)
        if not coords:
            print(f"[{key}] 無有效點位，跳過。")
            continue
        print(f"[{key}] 點數量：{len(coords)}，CRS：{point_crs} (EPSG={epsg})")

        # ❶ 每一回合各自建 SamGeo2（避免狀態殘留）
        sam = SamGeo2(model_id=model_id, automatic=False)
        sam.set_image(ortho)

        mask_path = os.path.join(mask_dir, f"mask_{key}.tif")

        try:
            # 先用「非批次」格式（你的原始寫法）
            sam.predict_by_points(
                point_coords_batch=coords,   # [[x,y], ...]
                point_crs=point_crs,
                output=mask_path,
                dtype="uint8",
            )
        except Exception as e:
            msg = str(e)
            # ❷ 若遇到 batched 錯誤，改用 batched 形式再試一次
            if "only be used when in batched mode" in msg:
                print(f"[{key}] 偵測到 batched 模式錯誤，改用 batched 輸入重試...")
                # 重新建一個，保險
                if sam is not None:
                    del sam
                sam = SamGeo2(model_id=model_id, automatic=False)
                sam.set_image(ortho)
                sam.predict_by_points(
                    point_coords_batch=[coords],  # [[[x,y], ...]] ← 多包一層
                    point_crs=point_crs,
                    output=mask_path,
                    dtype="uint8",
                )
            else:
                raise

        print(f"[{key}] 已輸出遮罩：{mask_path}")

        # 後續流程保持你原本寫法
        shpraw_name = f"sam_shpraw_Z{int(key)}"
        polys_fc = raster_to_polygon(mask_path, poly_gdb, shpraw_name)
        print(f"[{key}] RasterToPolygon 輸出：{polys_fc}")

        filtered_name = f"sam_filtered_Z{int(key)}"
        filtered_fc = os.path.join(filtered_gdb, filtered_name)
        spatial_join_keep_polys(polys_fc, points_fc, filtered_fc)
        print(f"[{key}] SpatialJoin 完成：{filtered_fc}")

        deleted_small, deleted_max = delete_small_and_largest(filtered_fc, small_area_threshold)
        print(f"[{key}] 已刪除小面積 {deleted_small} 筆，刪除最大面積 {deleted_max} 筆。")

        noholes_name = f"sam_noholes_Z{int(key)}"
        noholes_fc = os.path.join(filtered_gdb, noholes_name)
        eliminate_holes(filtered_fc, noholes_fc, hole_area_threshold)
        print(f"[{key}] 填洞完成（< {hole_area_threshold} 的洞）：{noholes_fc}")

        print(f"[{key}] ✅ 完成")

    except Exception as e:
        print(f"[{key}] ❌ 發生錯誤：{e}")
        # 想看堆疊可開啟：
        # print(traceback.format_exc())
    finally:
        # ✅ 無論成功或失敗，都釋放顯卡記憶體
        try:
            if sam is not None:
                del sam
        except Exception:
            pass
        gc.collect()
        if torch is not None and hasattr(torch, "cuda") and torch.cuda.is_available():
            try:
                torch.cuda.empty_cache()
            except Exception:
                pass

print("\n=== 全部序號處理結束 ===")


專案路徑： D:\ArcGIS\AI_training\AI_training.aprx
預設 gdb： D:\ArcGIS\AI_training\AI_training.gdb
專案資料夾： D:\ArcGIS\AI_training
目前 Map 名稱: Lizard
Map 空間參考: Transverse_Mercator 0
Python 可執行檔： C:\Program Files\ArcGIS\Pro\bin\ArcGISPro.exe
Conda 環境路徑： C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo
將處理以下配對序號： ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16']

========== 開始處理序號 01 ==========
影像： D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\ortho_splits\ortho_split_01.tif
點圖層： D:\ArcGIS\AI_training\split_points.gdb\T1
[01] 點數量：249，CRS：EPSG:32755 (EPSG=32755)


C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasin

[01] 已輸出遮罩：D:\ArcGIS\AI_training\masks\mask_01.tif
[01] RasterToPolygon 輸出：D:\ArcGIS\AI_training\geosam_output.gdb\sam_shpraw_Z1
[01] SpatialJoin 完成：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_filtered_Z1
[01] 已刪除小面積 12 筆，刪除最大面積 1 筆。
[01] 填洞完成（< 0.0005 的洞）：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_noholes_Z1
[01] ✅ 完成

========== 開始處理序號 02 ==========
影像： D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\ortho_splits\ortho_split_02.tif
點圖層： D:\ArcGIS\AI_training\split_points.gdb\T2
[02] 點數量：171，CRS：EPSG:32755 (EPSG=32755)


C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

[02] 已輸出遮罩：D:\ArcGIS\AI_training\masks\mask_02.tif
[02] RasterToPolygon 輸出：D:\ArcGIS\AI_training\geosam_output.gdb\sam_shpraw_Z2
[02] SpatialJoin 完成：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_filtered_Z2
[02] 已刪除小面積 1 筆，刪除最大面積 1 筆。
[02] 填洞完成（< 0.0005 的洞）：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_noholes_Z2
[02] ✅ 完成

========== 開始處理序號 03 ==========
影像： D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\ortho_splits\ortho_split_03.tif
點圖層： D:\ArcGIS\AI_training\split_points.gdb\T3
[03] 點數量：219，CRS：EPSG:32755 (EPSG=32755)


...

[14] 已輸出遮罩：D:\ArcGIS\AI_training\masks\mask_14.tif
[14] RasterToPolygon 輸出：D:\ArcGIS\AI_training\geosam_output.gdb\sam_shpraw_Z14
[14] SpatialJoin 完成：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_filtered_Z14
[14] 已刪除小面積 0 筆，刪除最大面積 1 筆。
[14] 填洞完成（< 0.0005 的洞）：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_noholes_Z14
[14] ✅ 完成

========== 開始處理序號 15 ==========
影像： D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\ortho_splits\ortho_split_15.tif
點圖層： D:\ArcGIS\AI_training\split_points.gdb\T15
[15] 點數量：69，CRS：EPSG:32755 (EPSG=32755)


C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

[15] 已輸出遮罩：D:\ArcGIS\AI_training\masks\mask_15.tif
[15] RasterToPolygon 輸出：D:\ArcGIS\AI_training\geosam_output.gdb\sam_shpraw_Z15
[15] SpatialJoin 完成：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_filtered_Z15
[15] 已刪除小面積 0 筆，刪除最大面積 1 筆。
[15] 填洞完成（< 0.0005 的洞）：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_noholes_Z15
[15] ✅ 完成

========== 開始處理序號 16 ==========
影像： D:\SfM_coral_reefs_product\Lizard_island\LIRS_competition_t1_DMW\Orthos\ortho_splits\ortho_split_16.tif
點圖層： D:\ArcGIS\AI_training\split_points.gdb\T16
[16] 點數量：99，CRS：EPSG:32755 (EPSG=32755)


C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
C:\Users\keelu\AppData\Local\ESRI\conda\envs\geo\Lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None

[16] 已輸出遮罩：D:\ArcGIS\AI_training\masks\mask_16.tif
[16] RasterToPolygon 輸出：D:\ArcGIS\AI_training\geosam_output.gdb\sam_shpraw_Z16
[16] SpatialJoin 完成：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_filtered_Z16
[16] 已刪除小面積 0 筆，刪除最大面積 1 筆。
[16] 填洞完成（< 0.0005 的洞）：D:\ArcGIS\AI_training\shp_filtered.gdb\sam_noholes_Z16
[16] ✅ 完成

=== 全部序號處理結束 ===
